In [ ]:
# IPython magic to import matplotlib and plot inline
%matplotlib inline

In [ ]:
# Path settings
import os
# we use a relative path here, you can also specify an absolute path for your system
out_path = "output/01_hyperelasticity"
os.makedirs(out_path, exist_ok=True)

# Hyperelasticity

Solves a simple mechanical problem with hyperelastic material model and demonstrates computation of stress, strain etc and export to ParaView.

In [ ]:
import fenics as fe
import os

In [ ]:
# mesh
mesh = fe.BoxMesh(fe.Point(0,0,0),fe.Point(10.0,1,1),100,10,10)
mesh

In [ ]:
# FunctionSpace
V = fe.VectorFunctionSpace(mesh, "Lagrange", 1)

In [ ]:
# Finite element functions
du = fe.TrialFunction(V)
v  = fe.TestFunction(V)
u  = fe.Function(V)

In [ ]:
# Mark boundary subdomians
left =  fe.CompiledSubDomain("near(x[0], side) && on_boundary", side = 0.0)

# Define Dirichlet BC: zero displacement on left side (x = 0)
c = fe.Expression(("0.0", "0.0", "0.0"), element=V.ufl_element())
bcl = fe.DirichletBC(V, c, left)
bcs = [bcl]


# Define Von Neuman BC: surface load on right side (x=10)
#-- a) function to define boundary
class RightBorder(fe.SubDomain):
    def inside(self, x, on_boundary):
        return x[0] >= 10.0-fe.DOLFIN_EPS and on_boundary
#-- b) mark up boundary in meshfunction
boundaries = fe.MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundaries.set_all(0)
right_border = RightBorder()
right_border.mark(boundaries, 1)

#-- c) compute surface load over buondary
dsp    = fe.Measure('ds', domain=mesh, subdomain_data=boundaries)
load_s = fe.Constant((100., 0., 0.))
# term in variational form
integral_S = fe.inner(load_s, u) * dsp(1)


#-- save boundary to vtu for inspection
boundary_file = fe.File(os.path.join(out_path, "boundaries.pvd"))
boundary_file << boundaries

In [ ]:
# Functions for 

def defGrad(u):
    """
    Compute deformation gradient from displacements
    """
    d = u.geometric_dimension()
    I = fe.Identity(d)
    F = I + fe.grad(u)             
    return F


def strainEnergyDensityFunctionNeoHookean(F, Ey, nu):
    """
    Computes strain energy density of neo hookean material model in function o f
    - F: deformation gradient
    - Ey: young's modulus
    - nu: poisson ratio
    """
    mu    = Ey / (2.*(1+nu))
    lmbda = Ey*nu / ((1+nu)*(1-2*nu))
    C = F.T*F      # Right Cauchy-Green tensor
    I1 = fe.tr(C)  # Invariants 
    J = fe.det(F)
    return (mu/2)*(I1 - 3) - mu*fe.ln(J) + (lmbda/2)*(fe.ln(J))**2

In [ ]:

# create deformation gradient
F = defGrad(u)

F = fe.variable(F) # !!! needed to be able to differentiate strain energydensity function wrt F for stress computation

# instantiate strain energy density function for given material properties
psi = strainEnergyDensityFunctionNeoHookean(F=F, Ey=1000, nu=0.4)

# body force
B = fe.Constant((0., 0., 0.))

# total potential energy 
Pi = psi * fe.dx - fe.inner(B, u) * fe.dx - integral_S

# Compute 1st variation of Pi (directional derivative about u in dir. of v)
Fpi = fe.derivative(Pi, u, v)

# Compute Jacobian of F
Jac = fe.derivative(Fpi, u, du)

In [ ]:
# Define the solver
problem = fe.NonlinearVariationalProblem(Fpi, u, bcs, Jac)
solver = fe.NonlinearVariationalSolver(problem)

# Set solver parameters (optional)
prm = solver.parameters
prm['nonlinear_solver'] = 'newton'
prm['newton_solver']['linear_solver'] = 'petsc'

prm['newton_solver']['error_on_nonconvergence'] = True
prm['newton_solver']['absolute_tolerance'] = 1E-9
prm['newton_solver']['relative_tolerance'] = 1E-8
prm['newton_solver']['maximum_iterations'] = 25
prm['newton_solver']['relaxation_parameter'] = 1.0

prm['newton_solver']['lu_solver']['report'] = True
#prm['newton_solver']['lu_solver']['reuse_factorization'] = False
#prm['newton_solver']['lu_solver']['same_nonzero_pattern'] = False
prm['newton_solver']['lu_solver']['symmetric'] = False

prm['newton_solver']['krylov_solver']['error_on_nonconvergence'] = True
prm['newton_solver']['krylov_solver']['absolute_tolerance'] = 1E-7
prm['newton_solver']['krylov_solver']['relative_tolerance'] = 1E-5
prm['newton_solver']['krylov_solver']['maximum_iterations'] = 1000
prm['newton_solver']['krylov_solver']['nonzero_initial_guess'] = True
if prm['newton_solver']['linear_solver'] == 'gmres':
    prm['newton_solver']['preconditioner'] = 'ilu'

# solve
solver.solve()

In [ ]:
# Save results

path_to_displ = os.path.join(out_path, "displacement.pvd")
print("Saving displacement solution to file. '%s'"%path_to_displ)
u.rename("displacement", "")
uViewer = fe.File(path_to_displ)
uViewer << u

In [ ]:
# compute max / min displacement
W = fe.FunctionSpace(mesh, 'P', 1)

u_magnitude = fe.sqrt(fe.dot(u, u))
u_magnitude = fe.project(u_magnitude, W)
print('Min/Max displacement:',
      u_magnitude.vector().get_local().min(),
      u_magnitude.vector().get_local().max())

In [ ]:
# displacement magnitude at some random point in domain
u_magnitude(fe.Point(9.2342, 0.234, 0.7364))

In [ ]:
Z = fe.TensorFunctionSpace(mesh, 'P', 1)

# Computation of the stresses
path_to_stress = os.path.join(out_path, "stress.pvd")
print("Stress derivation and saving to file '%s'"%path_to_displ)
S = fe.diff(psi, F)            # compute stress by differentiation of psi!
S_project = fe.project(S, Z)
S_project.rename("stress", "")
sigmaViewer = fe.File(path_to_stress)
sigmaViewer << S_project


In [ ]:
# Computation of van Mises Stress
s = S - (1./3) * fe.tr(S) * fe.Identity(u.geometric_dimension())
von_Mises = fe.sqrt( 3./2 * fe.inner(s, s) )
von_Mises_project = fe.project(von_Mises, W)
von_Mises_project.rename("van_mises", "")
path_to_stress = os.path.join(out_path, "stress_van_mises.pvd")
misesViewer = fe.File(path_to_stress)
misesViewer << von_Mises_project
print("Maximum equivalent stress:", von_Mises_project.vector().get_local().max())

In [ ]:
# a more compact way of writing all results into the same file:

xdmffile = fe.XDMFFile(os.path.join(out_path, 'results.xdmf'))
xdmffile.parameters["flush_output"] = True
xdmffile.parameters["functions_share_mesh"] = True

xdmffile.write(S_project,0)
xdmffile.write(von_Mises_project,0)
xdmffile.write(u,0)

xdmffile.close()